In [ ]:
程序以读入p.inp为例，inp为ABAQUS的模型文件，包含三角形单元顶点编号及坐标，具体格式请参见官方文档
编写python程序check_inp进行检测：
1.对顶点坐标、编号和面编号、三顶点编号的唯一性、范围进行检测，inp文件不需满足编号连续性，对重复的应进行删除或重编号
2.统计单元边的共享单元数，对于FEM模型，任何一条边都应被有且只有两个单元共享。此蛋白质模型出现了四个单元共享一条边的几何错误，这里将该边平移少许距离作为第二条新边，其中两个单元随其移动，定义并更新相应坐标和编号。
3.最后检测三维三角形单元是否交叉，对于有共享顶点的两单元，判断该点对边是否与另一单元相交，方法见后，若不相交则是正常情况不考虑；而对于有共享边的两单元，若其相对顶点在另一单元所在平面内且与另一相对顶点在共享边同侧，则为异常交叉情况；对于其他情况，则用通用的三维三角形相交算法[16]：
不失一般性，设拟相交判断的2个三角面片为T1（A1B1C1）和T2（A2B2C2），T1的3个顶点分别为A1、B1和C1，所在平面为π1，法向量为n1; T2的3个顶点分别为A2、B2和C2，所在平面为π2，法向量为n2。
(1)计算T1各顶点与π1的关系及T2各顶点与π2的关系，排除不相交情况。总流程的伪代码为:
if(A1、B1和C1在π2同侧或A2、B2和C2在π1同侧)
如：判断T2的3个顶点是否在平面π1同侧，建立表达式：t1=A1A2⋅n1,t2=A1B2⋅n1,t3=A1C2⋅n1.如果t1、t2和t3同号且都不为零，则三角面片T1和T2不相交。（有可能有一两个为0，其余同号，此时T1和T2可能不相交、相交、平行不相交，但一定异面，可在后面检测出）
else
  if(n1与n2平行，且T2任一顶点在π1上)
判定T1、T2共面，goto(2)；如果f=n1×n2=0，T1与T2是否平行(或共面)需要继续判断T2的任一顶点是否在π1上，即式(1) 中t1、t2和t3为0，则T1和T2共面；不在就T1和T2不相交
  else
判定T1、T2异面，goto(3)；如果f的计算结果不为0，则三角面片T1和T2异面
  end if
end if
(2)当T1、T2共面时，用python的shapely库（可判断二维三角形相交）判断T1和T2是否相交
(3)当T1、T2异面时，分别判断线段A1B1、B1C1、C1A1是否与三角面片T2相交，以及线段A2B2、B2C2、C2A2和三角面片是否与T1相交，如果以上判断结果均为否，则三角面片T1和T2不相交；如果以上判断结果有一个为真，则三角面片T1和T2相交。
以判断T2的边A2B2是否与三角面片T1相交为例：S1=A1A2·n1，S2=A1B2·n1.当S1=S2=0时，线段A2B2与T1共面；否则A2B2与T1异面。以下从2种情况进行讨论：
1)异面。由于A2B2与T1异面，首先计算(S1·S2)，倘若结果为正，则A2B2在π1同侧，判定A2B2不与T1相交；否则A2、B2在π1异侧且A2和B2至少有一点不在平面π1上。假设A2不在π1上。建立表达式：
g1=[A1B2⋅(A1B1×A1A2)]⋅[A1C1⋅(A1B1×A1A2)]
g2=[B1B2⋅(B1C1×B1A2)]⋅[B1A1⋅(B1C1×B1A2)]
g3=[C1B2⋅(C1A1×C1A2)]⋅[C1B1⋅(C1A1×C1A2)]
如果g1、g2和g3计算结果中存在小于0的情况，则线段A2B2与三角面片T1不相交；如果3个结果均为非负，则线段A2B2与三角面片T1相交
2)共面。如果A2B2与T1共面，用python的shapely库判断T1和A2B2是否相交
如果A2B2与T1共面，首先运用前述方法判断A2、B2是否在T1内，若至少存在一点在T1内，则线段A2B2与T1相交；若A2、B2都不在T1内，只有2种情况：A2B2与T1不相交和A2B2与T1共面相交。此时如果A2B2与T1共面相交，T1中必定存在边与T2异面相交，由于T1、T2的每一条边都要作相交判断，所以当A2、B2都不在T1内，算法中可认定为不相交或不作处理，不影响T1、T2相交判定最终结果。
在实验过程中，numpy及python默认小数类型在进行向量叉积、点积计算时不满足精度要求，导致程序误判，因此换成sympy中的有理数类型表示一切小数，但是大大增加了运行时间，一共运行了近两天，最后发现整个蛋白质模型共四处交叉异常情况，这里将每一处嵌入的点坐标进行微调，使其满足FEM单元要求。
    from shapely.geometry import LineString,Polygon
    import sympy as sp  #python自带和numpy及其不准确，几位数的小数运算算不对，但sympy太耗时了，这个用了两天
    
    aa=0
    
    def edge_intersect(edgepoint1,edgepoint2,tripoint1,tripoint2,tripoint3):
        global aa
        n1=(tripoint2-tripoint1).cross(tripoint3-tripoint1)
        if n1.dot(tripoint2-tripoint1)!=0 or n1.dot(tripoint3-tripoint1)!=0:
            print("n is inaccurate")
            aa=1
        s1=(edgepoint1-tripoint1).dot(n1)
        s2=(edgepoint2-tripoint1).dot(n1)
        if (s1==0 and s2==0):
            e1=[edgepoint1[0].evalf(),edgepoint1[1].evalf(),edgepoint1[2].evalf()]
            e2=[edgepoint2[0].evalf(),edgepoint2[1].evalf(),edgepoint2[2].evalf()]
            t1=[tripoint1[0].evalf(),tripoint1[1].evalf(),tripoint1[2].evalf()]
            t2=[tripoint2[0].evalf(),tripoint2[1].evalf(),tripoint2[2].evalf()]
            t3=[tripoint3[0].evalf(),tripoint3[1].evalf(),tripoint3[2].evalf()]
            line_coords = [(e1[0], e1[1],e1[2]),(e2[0], e2[1],e2[2])]
            triangle_coords = [(t1[0],t1[1],t1[2]), (t2[0],t2[1],t2[2]), (t3[0],t3[1],t3[2])]
            line = LineString(line_coords)
            triangle = Polygon(triangle_coords)
            return line.intersects(triangle)
        if(s1*s2>0):
            return False
        if(s1!=0):
            n_1=(tripoint2-tripoint1).cross(edgepoint1-tripoint1)
            if n_1.dot(tripoint2-tripoint1)!=0 or n_1.dot(edgepoint1-tripoint1)!=0:
             print("n_1 is inaccurate")
             aa=1
            g1=(edgepoint2-edgepoint1).dot(n_1)*(tripoint3-edgepoint1).dot(n_1)
            if g1<0:
                return False
            n_2=(tripoint3-tripoint2).cross(edgepoint1-tripoint2)
            if n_2.dot(tripoint3-tripoint2)!=0 or n_2.dot(edgepoint1-tripoint2)!=0:
             print("n_2 is inaccurate")
             aa=1
            g2=(edgepoint2-edgepoint1).dot(n_2)*(tripoint1-edgepoint1).dot(n_2)
            if g2<0:
                return False
            n_3=(tripoint1-tripoint3).cross(edgepoint1-tripoint3)
            if n_3.dot(tripoint1-tripoint3)!=0 or n_3.dot(edgepoint1-tripoint3)!=0:
             print("n_3 is inaccurate")
             aa=1
            g3=(edgepoint2-edgepoint1).dot(n_3)*(tripoint2-edgepoint1).dot(n_3)
            if g3<0:
                return False
            return True
        if (s1!=0 or s2==0):
            print("wrong1")
        n_1=(tripoint2-tripoint1).cross(edgepoint2-tripoint1)
        if (n_1).dot(tripoint2-tripoint1)!=0 or n_1.dot(edgepoint2-tripoint1)!=0:
             print("n_11 is inaccurate")
             aa=1
        g1=(edgepoint1-edgepoint2).dot(n_1)*(tripoint3-edgepoint2).dot(n_1)
        if g1<0:
            return False
        n_2=(tripoint3-tripoint2).cross(edgepoint2-tripoint2)
        if n_2.dot(tripoint3-tripoint2)!=0 or n_2.dot(edgepoint2-tripoint2)!=0:
             print("n_22 is inaccurate")
             aa=1
        g2=(edgepoint1-edgepoint2).dot(n_2)*(tripoint1-edgepoint2).dot(n_2)
        if g2<0:
            return False
        n_3=(tripoint1-tripoint3).cross(edgepoint2-tripoint3)
        if n_3.dot(tripoint1-tripoint3)!=0 or n_3.dot(edgepoint2-tripoint3)!=0:
             print("n_33 is inaccurate")
             aa=1
        g3=(edgepoint1-edgepoint2).dot(n_3)*(tripoint2-edgepoint2).dot(n_3)
        if g3<0:
            return False
        return True
            
    def _3dtriangle_intersect(tri1point1,tri1point2,tri1point3,tri2point1,tri2point2,tri2point3):
        global aa
        A1=sp.Matrix([sp.Rational(tri1point1[0]).limit_denominator(10**15),sp.Rational(tri1point1[1]).limit_denominator(10**15),sp.Rational(tri1point1[2]).limit_denominator(10**15)])
        B1=sp.Matrix([sp.Rational(tri1point2[0]).limit_denominator(10**15),sp.Rational(tri1point2[1]).limit_denominator(10**15),sp.Rational(tri1point2[2]).limit_denominator(10**15)])
        C1=sp.Matrix([sp.Rational(tri1point3[0]).limit_denominator(10**15),sp.Rational(tri1point3[1]).limit_denominator(10**15),sp.Rational(tri1point3[2]).limit_denominator(10**15)])
        A2=sp.Matrix([sp.Rational(tri2point1[0]).limit_denominator(10**15),sp.Rational(tri2point1[1]).limit_denominator(10**15),sp.Rational(tri2point1[2]).limit_denominator(10**15)])
        B2=sp.Matrix([sp.Rational(tri2point2[0]).limit_denominator(10**15),sp.Rational(tri2point2[1]).limit_denominator(10**15),sp.Rational(tri2point2[2]).limit_denominator(10**15)])
        C2=sp.Matrix([sp.Rational(tri2point3[0]).limit_denominator(10**15),sp.Rational(tri2point3[1]).limit_denominator(10**15),sp.Rational(tri2point3[2]).limit_denominator(10**15)])
        n1=(B1-A1).cross(C1-A1)
        if n1.dot(B1-A1)!=0 or n1.dot(C1-A1)!=0:
             print("n1 is inaccurate")
             aa=1
        n2=(B2-A2).cross(C2-A2)
        if n2.dot(B2-A2)!=0 or n2.dot(C2-A2)!=0:
             print(" A2 "+str(tri2point1)+" "+str(A2)+"\nB2 "+str(tri2point2)+" "+str(B2)+"\nC2 "+str(tri2point3)+" "+str(C2))
             print("n2 is inaccurate"+"\nA2B2 "+str(B2-A2)+"\nA2C2 "+str(C2-A2)+"\nn2 "+str(n2)+"\nn2.dot(B2-A2) "+str(n2.dot(B2-A2))+"\nn2.dot(C2-A2) "+str(n2.dot(C2-A2)))
             aa=1
        t1=(A2-A1).dot(n1)
        t2=(B2-A1).dot(n1)
        t3=(C2-A1).dot(n1)
        poly1 = Polygon([(tri1point1[0], tri1point1[1], tri1point1[2]), (tri1point2[0], tri1point2[1], tri1point2[2]), (tri1point3[0], tri1point3[1], tri1point3[2])])
        poly2 = Polygon([(tri2point1[0], tri2point1[1], tri2point1[2]), (tri2point2[0], tri2point2[1], tri2point2[2]), (tri2point3[0], tri2point3[1], tri2point3[2])])
        if((t1>0 and t2>0 and t3>0)or(t1<0 and t2<0 and t3<0)):
            return False
        if(n1.cross(n2)).is_zero and t1==0:
            if(t2!=0 or t3!=0):
                print("wrong2")   
            if poly1.intersects(poly2):
                 return True
            return False
        if(n1.cross(n2)).is_zero:
            return False
        if(edge_intersect(A1,B1,A2,B2,C2)):
            print(str(tri1point1)+" "+str(tri1point2))
            return True
        if(edge_intersect(B1,C1,A2,B2,C2)):
            print(str(tri1point2)+" "+str(tri1point3))
            return True
        if(edge_intersect(C1,A1,A2,B2,C2)):
            print(str(tri1point1)+" "+str(tri1point3))
            return True
        if(edge_intersect(A2,B2,A1,B1,C1)):
            print(str(tri2point1)+" "+str(tri2point2))
            return True
        if(edge_intersect(B2,C2,A1,B1,C1)):
            print(str(tri2point2)+" "+str(tri2point3))
            return True
        if(edge_intersect(C2,A2,A1,B1,C1)):
            print(str(tri2point1)+" "+str(tri2point3))
            return True
        return False
    
    def point_tri_intersect(point,tripoint1_common,tripoint2_common,tripoint3): 
        n=(tripoint2_common-tripoint1_common).cross(tripoint3-tripoint1_common)
        global aa
        if n.dot(tripoint2_common-tripoint1_common)!=0 or n.dot(tripoint3-tripoint1_common)!=0:
             print("nn is inaccurate")
             aa=1
        if n.dot(point-tripoint1_common)==0:
            nn=(tripoint2_common-tripoint1_common).cross(point-tripoint1_common)
            if nn.dot(tripoint2_common-tripoint1_common)!=0 or nn.dot(point-tripoint1_common)!=0:
             print("nnn is inaccurate")
             aa=1
            if nn.dot(n)==0:
                print("wrong3")
            return nn.dot(n)>0
        return False
        
    
    faces = []  #用于存储面的列表

    #读取inp文件
    i=0
    j=0
    node=0
    max=0
    min=1
    vertices_set = []
    intmax=0
    decimalmax=0
    with open("p.inp", 'r') as file:
        for line in file:
            i=i+1
            if(i<4):
                continue
            if line.startswith("*ELEMENT"):
                j=1
                continue
            parts = line.split(",")
            if len(parts) == 0:
                continue     
            if j==0:
                for number_str in [parts[1],parts[2],parts[3]]:
                    if '.' in number_str:
                    # 切分整数部分和小数部分
                      integer_part, decimal_part = number_str.split('.')
                    # 整数位数和小数位数
                      if len(integer_part)>intmax:
                            intmax=len(integer_part)
                      if len(decimal_part)>decimalmax:
                            decimalmax=len(decimal_part)
                #检查顶点是否重复
                vertex = (float(parts[1]), float(parts[2]), float(parts[3]))
                for v in vertices_set:
                    if v==vertex:
                     print("Error: Vertex {} same as Vertex {}.".format(parts[0],vertices_set.index(v)+1))
                    #break
                vertices_set.append(vertex)        
                node=node+1
            if j == 1:
                #检查顶点是否按逆时针顺序出现（需要？）
                v1, v2, v3 = int(parts[1]), int(parts[2]), int(parts[3])
                if(v1==v2 and v1==v3 and v2==v3):
                    print("Error: face index {} nodes wrong.".format(parts[0]))
                faces.append((parts[0],v1, v2, v3))
                if(v1>max): 
                    max=v1
                if(v1<min): 
                    min=v1
                if(v2>max): 
                    max=v2
                if(v2<min): 
                    min=v2
                if(v3>max): 
                    max=v3
                if(v3<min): 
                    min=v3
        if(min!=1 or max!=node): 
            print("Error faces {} {} {}".format(min,max,node))
        print("intmax "+str(intmax)+" decimalmax "+str(decimalmax))
 
        # for face in faces:
        #     for other_face in faces:
        #         if (face[1] in other_face) and (face[2] in other_face) and (face[3] in other_face) and (face[0]!=other_face[0]):
        #          print("Error: face {} same as face {}.".format(face,other_face))

        #检查任何单元的边是多少单元的公共边
#         for face in faces:
#             count1 = 0
#             count2 = 0
#             count3 = 0
#             otherface1=[]
#             otherface2=[]
#             otherface3=[]
#             for other_face in faces:
#                 o=[other_face[1],other_face[2],other_face[3]]
#                 if face != other_face and (face[1] in other_face) and (face[2] in other_face):
#                     count1=count1+1
#                     otherface1.append(other_face)
#                 if face != other_face and (face[1] in other_face) and (face[3] in other_face):
#                     count2=count2+1
#                     otherface2.append(other_face)
#                 if face != other_face and (face[2] in other_face) and (face[3] in other_face):
#                     count3=count3+1
#                     otherface3.append(other_face)
#             if count1 != 1 or count2!=1 or count3!=1:
#                 print("Error: Face {} is not shared by exactly 2 faces.".format(face))
#                 print(str(count1)+" "+str(otherface1))
#                 print(str(count2)+" "+str(otherface2))
#                 print(str(count3)+" "+str(otherface3))

        #检查任一两面在3D空间是否重叠   
        for index, face in enumerate(faces):
            for face1 in faces[index+1:]:
               aa=0
#                if face1==face:   
#                     continue      
               if face[1] in face1:
                    if face[2] in face1:
                       p = sp.Matrix([sp.Rational((vertices_set[face[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[2]).limit_denominator(10**15)])
                       c=[]
                       b=[]
                       for f0 in face1[1:]:
                        if f0==face[1] or f0==face[2]:
                           c.append(sp.Matrix([sp.Rational((vertices_set[f0-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[2]).limit_denominator(10**15)]))
                           continue
                        b.append(sp.Matrix([sp.Rational((vertices_set[f0-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[2]).limit_denominator(10**15)]))
                       if len(c)!=2 or len(b)!=1:
                            print("wrong "+str(face)+" "+str(face1))
                       if point_tri_intersect(p,c[0],c[1],b[0]):  
                        print("Triangles overlap:"+str(face)+" "+str(face1))
                       continue
                    if face[3] in face1:
                       p = sp.Matrix([sp.Rational((vertices_set[face[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[2]).limit_denominator(10**15)])
                       c=[]
                       b=[]
                       for f0 in face1[1:]:
                        if f0==face[1] or f0==face[3]:
                           c.append(sp.Matrix([sp.Rational((vertices_set[f0-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[2]).limit_denominator(10**15)]))
                           continue
                        b.append(sp.Matrix([sp.Rational((vertices_set[f0-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[2]).limit_denominator(10**15)]))
                       if len(c)!=2 or len(b)!=1:
                            print("wrong_1 "+str(face)+" "+str(face1))
                       if point_tri_intersect(p,c[0],c[1],b[0]):  
                        print("Triangles overlap:"+str(face)+" "+str(face1))
                       continue
                    if face[2] in face1 or face[3] in face1:
                        print("wrong_2 "+str(face)+" "+str(face1))
                    e1=sp.Matrix([sp.Rational((vertices_set[face[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[2]).limit_denominator(10**15)])
                    e2=sp.Matrix([sp.Rational((vertices_set[face[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[2]).limit_denominator(10**15)])
                    f1p1=sp.Matrix([sp.Rational((vertices_set[face1[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[1]-1])[2]).limit_denominator(10**15)])
                    f1p2=sp.Matrix([sp.Rational((vertices_set[face1[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[2]-1])[2]).limit_denominator(10**15)])
                    f1p3=sp.Matrix([sp.Rational((vertices_set[face1[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[3]-1])[2]).limit_denominator(10**15)])
                    if edge_intersect(e1,e2,f1p1,f1p2,f1p3): 
                        print("Triangles overlap:"+str(face)+" "+str(face1))
                        continue
                    e=[]
                    for f in face1[1:]:
                        if f==face[1]:
                            continue
                        e.append(sp.Matrix([sp.Rational((vertices_set[f-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f-1])[2]).limit_denominator(10**15)]))
                    if len(e)!=2:
                        print("wrong_3 "+str(face)+" "+str(face1))
                    f2p1=sp.Matrix([sp.Rational((vertices_set[face[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[2]).limit_denominator(10**15)])
                    f2p2=sp.Matrix([sp.Rational((vertices_set[face[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[2]).limit_denominator(10**15)])
                    f2p3=sp.Matrix([sp.Rational((vertices_set[face[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[2]).limit_denominator(10**15)])
                    if edge_intersect(e[0],e[1],f2p1,f2p2,f2p3):  
                        print("Triangles overlap:"+str(face)+" "+str(face1)) 
                    continue
               if face[1] in face1:
                        print("wrong_4 "+str(face)+" "+str(face1))
               if face[2] in face1:
                   if face[3] in face1:
                       p = sp.Matrix([sp.Rational((vertices_set[face[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[2]).limit_denominator(10**15)])
                       c=[]
                       b=[]
                       for f0 in face1[1:]:
                        if f0==face[3] or f0==face[2]:
                           c.append(sp.Matrix([sp.Rational((vertices_set[f0-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[2]).limit_denominator(10**15)]))
                           continue
                        b.append(sp.Matrix([sp.Rational((vertices_set[f0-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f0-1])[2]).limit_denominator(10**15)]))
                       if len(c)!=2 or len(b)!=1:
                            print("wrong_5 "+str(face)+" "+str(face1))
                       if point_tri_intersect(p,c[0],c[1],b[0]):  
                        print("Triangles overlap:"+str(face)+" "+str(face1))
                       continue    
                   e1=sp.Matrix([sp.Rational((vertices_set[face[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[2]).limit_denominator(10**15)])
                   e2=sp.Matrix([sp.Rational((vertices_set[face[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[2]).limit_denominator(10**15)])
                   f1p1=sp.Matrix([sp.Rational((vertices_set[face1[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[1]-1])[2]).limit_denominator(10**15)])
                   f1p2=sp.Matrix([sp.Rational((vertices_set[face1[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[2]-1])[2]).limit_denominator(10**15)])
                   f1p3=sp.Matrix([sp.Rational((vertices_set[face1[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[3]-1])[2]).limit_denominator(10**15)])
                   if edge_intersect(e1,e2,f1p1,f1p2,f1p3):
                        print("Triangles overlap:"+str(face)+" "+str(face1))
                        continue
                   e=[]
                   for f in face1[1:]:
                        if f==face[2]:
                            continue
                        e.append(sp.Matrix([sp.Rational((vertices_set[f-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f-1])[2]).limit_denominator(10**15)]))
                   if len(e)!=2:
                        print("wrong_6 "+str(face)+" "+str(face1))
                   f2p1=sp.Matrix([sp.Rational((vertices_set[face[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[2]).limit_denominator(10**15)])
                   f2p2=sp.Matrix([sp.Rational((vertices_set[face[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[2]).limit_denominator(10**15)])
                   f2p3=sp.Matrix([sp.Rational((vertices_set[face[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[2]).limit_denominator(10**15)])
                   if edge_intersect(e[0],e[1],f2p1,f2p2,f2p3):  
                        print("Triangles overlap:"+str(face)+" "+str(face1)) 
                   continue
               if face[2] in face1:
                        print("wrong_7 "+str(face)+" "+str(face1))
               if face[3] in face1:
                   e1=sp.Matrix([sp.Rational((vertices_set[face[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[2]).limit_denominator(10**15)])
                   e2=sp.Matrix([sp.Rational((vertices_set[face[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[2]).limit_denominator(10**15)])
                   f1p1=sp.Matrix([sp.Rational((vertices_set[face1[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[1]-1])[2]).limit_denominator(10**15)])
                   f1p2=sp.Matrix([sp.Rational((vertices_set[face1[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[2]-1])[2]).limit_denominator(10**15)])
                   f1p3=sp.Matrix([sp.Rational((vertices_set[face1[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face1[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face1[3]-1])[2]).limit_denominator(10**15)])
                   if edge_intersect(e1,e2,f1p1,f1p2,f1p3):
                        print("Triangles overlap:"+str(face)+" "+str(face1))
                        continue
                   e=[]
                   for f in face1[1:]:
                        if f==face[3]:
                            continue
                        e.append(sp.Matrix([sp.Rational((vertices_set[f-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[f-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[f-1])[2]).limit_denominator(10**15)]))
                   if len(e)!=2:
                        print("wrong_8 "+str(face)+" "+str(face1))
                   f2p1=sp.Matrix([sp.Rational((vertices_set[face[1]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[1]-1])[2]).limit_denominator(10**15)])
                   f2p2=sp.Matrix([sp.Rational((vertices_set[face[2]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[2]-1])[2]).limit_denominator(10**15)])
                   f2p3=sp.Matrix([sp.Rational((vertices_set[face[3]-1])[0]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[1]).limit_denominator(10**15),sp.Rational((vertices_set[face[3]-1])[2]).limit_denominator(10**15)])
                   if edge_intersect(e[0],e[1],f2p1,f2p2,f2p3):  
                        print("Triangles overlap:"+str(face)+" "+str(face1)) 
                   continue
                
               if _3dtriangle_intersect(vertices_set[face[1]-1], vertices_set[face[2]-1], vertices_set[face[3]-1], vertices_set[face1[1]-1], vertices_set[face1[2]-1], vertices_set[face1[3]-1]):
                 print("Triangles overlap:"+str(face)+" "+str(face1))
               if(aa==1):
                print("inaccurate n:"+str(face)+" "+str(face1))
                    
    
    print("SMF file is valid.")
    

#正常的网格：任何单元的边都是且仅是两个不同的单元的公共边，任何单元除了此类公共边或公共顶点不能有其他公共部分，但如果两个体只是接触（没有嵌入）是否能画网格？
#p.inp:首先体封闭，无重复点
#     Error: Face ('3666', 3772, 1627, 873) is not shared by exactly 2 faces.
#     3 [('3673', 3772, 875, 1627), ('4616', 1627, 1339, 3772), ('4617', 1627, 3772, 1626)]
#     1 [('3667', 3772, 873, 4873)]
#     1 [('686', 1033, 873, 1627)]
#     Error: Face ('3673', 3772, 875, 1627) is not shared by exactly 3 other faces.
#     1 [('3672', 3772, 877, 875)]
#     3 [('3666', 3772, 1627, 873), ('4616', 1627, 1339, 3772), ('4617', 1627, 3772, 1626)]
#     1 [('687', 1036, 1627, 875)]
#     Error: Face ('4616', 1627, 1339, 3772) is not shared by exactly 2 faces.
#     1 [('4615', 1627, 1035, 1339)]
#     3 [('3666', 3772, 1627, 873), ('3673', 3772, 875, 1627), ('4617', 1627, 3772, 1626)]
#     1 [('4622', 1339, 876, 3772)]
#     Error: Face ('4617', 1627, 3772, 1626) is not shared by exactly 2 faces.
#     3 [('3666', 3772, 1627, 873), ('3673', 3772, 875, 1627), ('4616', 1627, 1339, 3772)]
#     1 [('684', 1626, 1033, 1627)]
#     1 [('4618', 1626, 3772, 1340)]
#node:1627,-17.7878,7.33922,-19.9076;3772,-17.0584,7.2356,-19.4247  新加node：4995,-17.7877,7.33932,-19.9075;4996,-17.0583,7.2357,-19.4246
#将以下10个面的3772点改为4996：
# ('3666', 3772, 1627, 873)
# ('3667', 3772, 873, 4873)
# ('4626', 4873, 1340, 3772)
# ('4618', 1626, 3772, 1340)
# ('4617', 1627, 3772, 1626)
#不用改：
# ('3673', 3772, 875, 1627)
# ('3672', 3772, 877, 875)
# ('3671', 876, 877, 3772)
# ('4622', 1339, 876, 3772)
# ('4616', 1627, 1339, 3772)
#将以下8个面的1627点改为4995：
# ('4617', 1627, 3772, 1626)
# ('684', 1626, 1033, 1627)
# ('686', 1033, 873, 1627)
# ('3666', 3772, 1627, 873)
#不用改：
# ('3673', 3772, 875, 1627)
# ('687', 1036, 1627, 875)
# ('685', 1035, 1627, 1036)
# ('4615', 1627, 1035, 1339)
# ('4616', 1627, 1339, 3772)
# intmax 3 decimalmax 8，改完面后：
# Triangles overlap:('12', 22, 24, 23) ('2697', 53, 3787, 24)
# Triangles overlap:('2694', 3787, 50, 23) ('2697', 53, 3787, 24)
# Triangles overlap:('2695', 23, 50, 22) ('2697', 53, 3787, 24)
# Triangles overlap:('3097', 394, 3874, 290) ('7370', 289, 392, 4926)
# Triangles overlap:('3098', 290, 3874, 392) ('7370', 289, 392, 4926)
# Triangles overlap:('3101', 395, 394, 290) ('7370', 289, 392, 4926)
# Triangles overlap:('3101', 395, 394, 290) ('7378', 290, 392, 289)
# Triangles overlap:('3666', 4996, 4995, 873) ('3671', 876, 877, 3772)
# Triangles overlap:('3667', 4996, 873, 4873) ('3671', 876, 877, 3772)
# Triangles overlap:('3671', 876, 877, 3772) ('4617', 4995, 4996, 1626)
# Triangles overlap:('3671', 876, 877, 3772) ('4618', 1626, 4996, 1340)
# Triangles overlap:('3671', 876, 877, 3772) ('4626', 4873, 1340, 4996)
# Triangles overlap:('3791', 4026, 4025, 986) ('3795', 4028, 4026, 989)
# Triangles overlap:('3791', 4026, 4025, 986) ('3797', 4027, 4028, 989)
# Triangles overlap:('3791', 4026, 4025, 986) ('3798', 4027, 989, 986)
# Triangles overlap:('4618', 1626, 4996, 1340) ('4622', 1339, 876, 3772)
# Triangles overlap:('4622', 1339, 876, 3772) ('4626', 4873, 1340, 4996)
# Triangles overlap:('6331', 3291, 4612, 3294) ('7015', 4795, 3700, 4427)
# Triangles overlap:('6331', 3291, 4612, 3294) ('7019', 4796, 4797, 4795)
# Triangles overlap:('6331', 3291, 4612, 3294) ('7021', 4795, 4797, 3700)
# Triangles overlap:('6336', 3294, 4612, 3090) ('7015', 4795, 3700, 4427)
# Triangles overlap:('6336', 3294, 4612, 3090) ('7016', 4795, 4427, 2454)
# Triangles overlap:('6336', 3294, 4612, 3090) ('7017', 4795, 2454, 2455)
# Triangles overlap:('6336', 3294, 4612, 3090) ('7019', 4796, 4797, 4795)
# Triangles overlap:('6336', 3294, 4612, 3090) ('7020', 4796, 4795, 2455)
#改成：
# 23,-20.2282,10.7391,9.83723
# 290,-13.0967,14.0656,21.4927
# 4995,-17.7877,7.33932,-19.9074
# 4996,-17.0583,7.2357,-19.4245
# 989,-11.992,-20.1387,15.1

intmax 3 decimalmax 8
Triangles overlap:('12', 22, 24, 23) ('2697', 53, 3787, 24)
Triangles overlap:('2694', 3787, 50, 23) ('2697', 53, 3787, 24)
Triangles overlap:('2695', 23, 50, 22) ('2697', 53, 3787, 24)
Triangles overlap:('3097', 394, 3874, 290) ('7370', 289, 392, 4926)
Triangles overlap:('3098', 290, 3874, 392) ('7370', 289, 392, 4926)
Triangles overlap:('3101', 395, 394, 290) ('7370', 289, 392, 4926)
Triangles overlap:('3101', 395, 394, 290) ('7378', 290, 392, 289)
Triangles overlap:('3666', 4996, 4995, 873) ('3671', 876, 877, 3772)
Triangles overlap:('3667', 4996, 873, 4873) ('3671', 876, 877, 3772)
Triangles overlap:('3671', 876, 877, 3772) ('4617', 4995, 4996, 1626)
Triangles overlap:('3671', 876, 877, 3772) ('4618', 1626, 4996, 1340)
Triangles overlap:('3671', 876, 877, 3772) ('4626', 4873, 1340, 4996)
Triangles overlap:('3791', 4026, 4025, 986) ('3795', 4028, 4026, 989)
Triangles overlap:('3791', 4026, 4025, 986) ('3797', 4027, 4028, 989)
Triangles overlap:('3791', 4026, 40